In [ ]:
from csc4585 import *

In [ ]:
%%writec pth0.cc
#include <thread>
#include <iostream>

int i=0;

void inc() {
  for(int j=0;j<1000;j++)
    i++;
  std::cout << "i=" << i << std::endl;
}

int main() {
  for(int j=0;j<100;j++) {
    // produces no output, program
    // exits before threads finish
    new std::thread(inc);
  }
  return 0;
}

In [ ]:
%%writec pth1.cc
#include <thread>
#include <iostream>

int i=0;

void inc() {
  for(int j=0;j<1000;j++)
    i++;
  std::cout << "i=" << i << std::endl;
}

int main() {
  for(int j=0;j<100;j++) {
    std::thread t(inc);
    // wait for the thread to finish right
    // after starting it. This essentially
    // runs seqeuentially.
    t.join();
  }
  return 0;
}

In [ ]:
%%writec pth2.cc
#include <thread>
#include <iostream>

int i=0;

void inc() {
  for(int j=0;j<1000;j++)
    i++;
  std::cout << "i=" << i << std::endl;
}

int main() {
  for(int j=0;j<100;j++) {
    std::thread t(inc);
    // program will wait for this thread
    t.detach();
  }
  return 0;
}

In [ ]:
%%writec pth5.cc
#include <thread>
#include <vector>
#include <iostream>

int i=0;

void inc() {
  for(int j=0;j<1000;j++)
    i++;
  std::cout << "i=" << i << std::endl;
}

int main() {
  std::vector<std::thread> vec;
  // start and add all threads to a vector
  for(int j=0;j<100;j++) {
    vec.push_back(std::thread(inc));
  }
  // join all threads in a vector
  for(auto j=vec.begin();j != vec.end();++j) {
    j->join();
  }
  return 0;
}

In [ ]:
%%writec pth6.cc
#include <thread>
#include <vector>
#include <iostream>
#include <mutex>

int i=0;

// This is new...
std::mutex m;

void inc() {
  for(int j=0;j<1000;j++) {
    m.lock(); // mutually exclusive access
    i++;
    m.unlock(); // release access
  }
  std::cout << "i=" << i << std::endl;
}

int main() {
  std::vector<std::thread> vec;
  for(int j=0;j<100;j++) {
    vec.push_back(std::thread(inc));
  }
  for(auto j=vec.begin();j != vec.end();++j) {
    j->join();
  }
  return 0;
}

In [ ]:
%%writec pth7.cc
#include <thread>
#include <vector>
#include <iostream>
#include <mutex>

int i=0;

std::mutex m;

void inc() {
  for(int j=0;j<1000;j++) {
    // The unlock is in the destructor
    std::lock_guard<std::mutex> lg(m);
    i++;
  }
  std::cout << "i=" << i << std::endl;
}

int main() {
  std::vector<std::thread> vec;
  for(int j=0;j<100;j++) {
    vec.push_back(std::thread(inc));
  }
  for(auto j=vec.begin();j != vec.end();++j) {
    j->join();
  }
  std::cout << "AT FINISH: " << i << std::endl;
  return 0;
}

In [ ]:
%%writec pth8.cc
#include <thread>
#include <vector>
#include <iostream>
#include <mutex>
#include <condition_variable>

int i=0;

std::mutex m;
std::condition_variable cv;
const int N = 100;
const int N_INC = 1000;

void inc() {
  {
    std::lock_guard<std::mutex> lg(m);
    for(int j=0;j<N_INC;j++) {
      i++;
    }
    std::cout << "i=" << i << std::endl;
  }
  // after unlock
  cv.notify_all();
}

void fin() {
  std::unique_lock<std::mutex> ul(m);

  // Wait for a condition
  cv.wait(ul,[](){ return i==N*N_INC; });
    
  std::cout << "AT FINISH: " << i << std::endl;
}

int main() {
  std::thread f(fin);
  std::vector<std::thread> vec;
  for(int j=0;j<N;j++) {
    vec.push_back(std::thread(inc));
  }
  for(auto j=vec.begin();j != vec.end();++j) {
    j->join();
  }
  f.join();
  return 0;
}

In [ ]:
%%writec pth9.cc
#include <thread>
#include <iostream>
#include <mutex>
#include <condition_variable>

int i=0;

std::mutex m;
std::condition_variable cv;

const int pki

void inc() {
  {
    std::lock_guard<std::mutex> lg(m);
    for(int j=0;j<N_INC;j++) {
      i++;
    }
    std::cout << "i=" << i << std::endl;
  }
  // after unlock
  cv.notify_all();
}

void fin() {
  std::unique_lock<std::mutex> ul(m);
  cv.wait(ul,[](){ return i==N_INC*N; });
}

int main() {
  std::thread f(fin);
  for(int j=0;j<N;j++) {
    std::thread t(inc);
    // detach most of the threads
    t.detach();
  }
  f.join();
  std::cout << "AT FINISH: " << i << std::endl;
  return 0;
}